In [138]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import re

file_path = 'test.txt'

# Чтение текста из файла и очистка от лишних символов
with open(file_path, 'r', encoding='utf-8-sig') as file:
    raw_text = file.read()

cleaned_text = re.sub(r'[^А-Яа-я ]', '', raw_text)
# Токенизация текста
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([cleaned_text])

vocab_size = len(tokenizer.word_index) + 1  # Общее количество уникальных символов
data_matrix = tokenizer.texts_to_matrix(cleaned_text)  # Преобразует исходный текст в массив

# Формирование входных и выходных данных для обучения
sequence_length = 6
num_samples = len(cleaned_text) - sequence_length
# X_train = np.array([tokenizer.texts_to_matrix(cleaned_text[i:i + sequence_length]) for i in range(num_samples)])
X_train = np.array([data_matrix[i:i + sequence_length, :] for i in range(num_samples)])
Y_train = data_matrix[sequence_length:]
# Y_train = data_matrix[sequence_length:]
# Определение параметров модели
model = Sequential([
    Input((sequence_length, vocab_size)),
    SimpleRNN(128, activation='tanh', return_sequences=True),
    SimpleRNN(128, activation='tanh'),
    Dense(vocab_size, activation='softmax')
])

optimizer = Adam()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

# Обучение модели
model.fit(X_train, Y_train, batch_size=128, epochs=230)

# for i in range(5):
#     print(f"Символ: {cleaned_text[i]}, One-hot encoding: {tokenizer.texts_to_matrix(cleaned_text[i])}")
# print(f"Размерности входных данных: {X_train.shape}")
# print(f"Размерности выходных данных: {Y_train.shape}")
# intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)
# intermediate_output = intermediate_layer_model.predict(X_train[:5])
# print(f"Промежуточный вывод для первых 5 примеров: {intermediate_output}")


Epoch 1/230
299/299 [==============================] - 10s 22ms/step - loss: 2.7644 - accuracy: 0.2162
Epoch 2/230
299/299 [==============================] - 6s 21ms/step - loss: 2.4419 - accuracy: 0.2742
Epoch 3/230
299/299 [==============================] - 6s 21ms/step - loss: 2.3483 - accuracy: 0.2941
Epoch 4/230
299/299 [==============================] - 8s 25ms/step - loss: 2.2835 - accuracy: 0.3133
Epoch 5/230
299/299 [==============================] - 8s 26ms/step - loss: 2.2230 - accuracy: 0.3293
Epoch 6/230
299/299 [==============================] - 8s 27ms/step - loss: 2.1708 - accuracy: 0.3451
Epoch 7/230
299/299 [==============================] - 8s 28ms/step - loss: 2.1215 - accuracy: 0.3590
Epoch 8/230
299/299 [==============================] - 9s 30ms/step - loss: 2.0749 - accuracy: 0.3736
Epoch 9/230
299/299 [==============================] - 9s 31ms/step - loss: 2.0334 - accuracy: 0.3836
Epoch 10/230
299/299 [==============================] - 10s 32ms/step - loss: 1.9

In [149]:
def generate_text(seed_text, model, tokenizer, vocab_size, seq_length=30):
    generated_text = seed_text

    for _ in range(seq_length):
        recent_chars_one_hot = []
        for char in generated_text[-sequence_length:]:
            one_hot_encoded_char = tokenizer.texts_to_matrix(char)  # преобразование символов в one-hot-encoding
            recent_chars_one_hot.append(one_hot_encoded_char)

        recent_chars_matrix = np.array(recent_chars_one_hot)
        input_sequence = recent_chars_matrix.reshape(1, sequence_length, vocab_size)  # формирование начальных символов

        pred = model.predict(input_sequence)  # предсказывание следующего символа
        predicted_char = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получение ответа в виде символа по словарю

        generated_text += predicted_char  # добавление символа в строку

    return generated_text

seed_text = "холодн"
generated_text = generate_text(seed_text, model, tokenizer, vocab_size, seq_length=30)
print(generated_text)



1/1 [==============================] - 0s 46ms/step
холодная в которого служить в долг д
